In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as mino
import sklearn
import pickle

In [23]:
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [27]:
%watermark -ajinhyo -u -d -v -pnumpy,pandas,matplotlib,seaborn,missingno,sklearn

jinhyo 
last updated: 2019-09-04 

CPython 3.7.3
IPython 7.4.0

numpy 1.16.2
pandas 0.24.2
matplotlib 3.0.3
seaborn 0.9.0
missingno 0.4.2
sklearn 0.20.3


---

# 데이터 불러오기

In [62]:
# 붙임4의 columns를 카피함.

names = ["year"]
names.extend(problem.columns)
names

['year',
 'doy',
 'h',
 'm',
 'Np',
 'Tp',
 'Vp',
 'B_gsm_x',
 'B_gsm_y',
 'B_gsm_z',
 'Bmag']

In [77]:
# 붙임1 지자기교란 예측모델 개발용 데이터셋_수정190820/태양풍

ace = {}
for y in range(1999, 2014):
    ace[y] = pd.read_csv(f"ace_{y}.csv", skiprows=15, delim_whitespace=True, names=names)

ace[1999].head(25)

,year,doy,h,m,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag
0,1999,1,0,0,7.149,92352.0,406.00,-2.174,-2.598,5.550,6.630
1,1999,1,0,1,5.998,85859.0,419.12,-1.245,-0.140,6.558,6.796
2,1999,1,0,2,6.211,81547.0,411.99,-2.003,-1.198,6.306,6.802
3,1999,1,0,3,6.680,72308.0,405.25,-3.093,-2.483,5.545,6.854
4,1999,1,0,4,-9999.900,-9999.9,-9999.90,-3.009,-1.500,5.908,6.842
5,1999,1,0,5,6.410,69888.0,408.27,-2.297,-0.685,6.339,6.842
6,1999,1,0,7,6.149,74253.0,409.85,-2.132,0.345,6.415,6.831
7,1999,1,0,8,6.422,73144.0,407.99,-2.416,0.694,6.021,6.969
8,1999,1,0,9,6.216,88505.0,412.83,-2.245,3.919,5.365,7.032
9,1999,1,0,10,5.587,86869.0,413.50,-2.461,3.513,5.546,7.028


In [66]:
# 붙임1 지자기교란 예측모델 개발용 데이터셋_수정190820/지자기교란 지수

target = pd.read_csv("kp_1999-2013.csv")
target.head()

,date,kp_0h,kp_3h,kp_6h,kp_9h,kp_12h,kp_15h,kp_18h,kp_21h
0,1999-01-01,0,2,1,2,2,1,1,1
1,1999-01-02,1,2,2,3,3,2,2,1
2,1999-01-03,2,2,0,0,1,1,1,1
3,1999-01-04,1,2,3,2,3,2,1,2
4,1999-01-05,3,3,2,3,1,1,2,1


In [159]:
target

,date,kp_0h,kp_3h,kp_6h,kp_9h,kp_12h,kp_15h,kp_18h,kp_21h
0,1999-01-01,0,2,1,2,2,1,1,1
1,1999-01-02,1,2,2,3,3,2,2,1
2,1999-01-03,2,2,0,0,1,1,1,1
3,1999-01-04,1,2,3,2,3,2,1,2
4,1999-01-05,3,3,2,3,1,1,2,1
5,1999-01-06,1,0,3,1,3,3,3,2
6,1999-01-07,1,0,1,2,3,3,2,2
7,1999-01-08,3,2,3,4,3,3,3,3
8,1999-01-09,3,2,3,1,1,1,1,2
9,1999-01-10,2,0,2,3,1,1,1,1


In [58]:
# 붙임2 AI 경진대회 문제(태양풍 데이터)

problem = pd.read_csv("problem.csv")
problem.head()

,doy,h,m,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag
0,1,0,0,1.852,41618.0,455.61,6.098,2.449,-13.415,14.952
1,1,0,1,2.181,33781.0,461.07,6.700,1.502,-13.232,14.912
2,1,0,2,2.415,34709.0,464.84,7.390,0.523,-12.941,14.928
3,1,0,3,2.359,34183.0,470.76,8.011,-0.098,-12.694,15.013
4,1,0,4,1.237,32467.0,469.51,7.493,0.410,-13.058,15.064


In [75]:
# 붙임5 추가 태양풍 데이터(과제문제의 전년도 6개월)

additional = pd.read_csv("additional.csv", skiprows=14, delim_whitespace=True, names=problem.columns)
additional.head()

,doy,h,m,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag
0,180,0,1,2.416,65701.0,402.18,-4.212,-3.982,2.445,6.298
1,180,0,2,2.493,68240.0,399.89,-4.566,-3.473,2.766,6.430
2,180,0,3,2.582,85224.0,389.17,-5.456,-2.370,2.837,6.596
3,180,0,4,2.686,90925.0,390.59,-5.550,-1.988,2.768,6.519
4,180,0,5,2.676,79821.0,392.44,-4.922,-3.073,2.679,6.416


---

# 데이터 전처리

In [92]:
from copy import deepcopy
ace_origin = deepcopy(ace)

In [116]:
# datetime 객체로 변환

pd.to_datetime(ace[2013]["year"][:5], format="%Y")

0   2013-01-01
1   2013-01-01
2   2013-01-01
3   2013-01-01
4   2013-01-01
Name: year, dtype: datetime64[ns]

In [117]:
# day, hour, minute을 더해주기 위해서 timedelta객체로 만듦

ace[2013]["doy"][:5].map(lambda x: pd.Timedelta(x-1, unit='days'))

0   0 days
1   0 days
2   0 days
3   0 days
4   0 days
Name: doy, dtype: timedelta64[ns]

In [146]:
%%time
# 연도가 1월 1일로 설정되어 있으므로 x-1을 함. 

for year in range(1999, 2014):
    ace[year]["dt"] = pd.to_datetime(ace[year]["year"], format="%Y") + \
    ace[year]["doy"].map(lambda x: pd.Timedelta(x-1, unit='days')) + \
    ace[year]["h"].map(lambda x: pd.Timedelta(x, unit='hours')) + \
    ace[year]["m"].map(lambda x: pd.Timedelta(x, unit='minutes'))

Wall time: 5min 15s


In [157]:
# 인덱스 변경

for year in range(1999,2014):
    ace[year] = ace[year].iloc[:,4:].set_index("dt")

In [155]:
# 저장

with open("ace_with_datetime.pickle", "wb") as fp:
    pickle.dump(ace, fp)

In [158]:
# 저장

for year in range(1999,2014):
    ace[year].to_csv(f"ace_{year}_datetime.csv")